<a href="https://colab.research.google.com/github/VikrantTufani1999/Solvathon_2023/blob/main/Random_Forest_Approach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
dataset = pd.read_csv('/processedDatSet1.csv')
dataset = dataset.fillna(0)
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

In [4]:
dataset.head()
print(dataset.isnull().any())



Exercise                        False
Depression                      False
Diabetes                        False
Arthritis                       False
Sex                             False
Age_Category                    False
Height_(cm)                     False
Weight_(kg)                     False
BMI                             False
Smoking_History                 False
Alcohol_Consumption             False
Fruit_Consumption               False
Green_Vegetables_Consumption    False
FriedPotato_Consumption         False
General_Health_Excellent        False
General_Health_Fair             False
General_Health_Good             False
General_Health_Poor             False
General_Health_Very Good        False
Heart_Disease                   False
dtype: bool


In [5]:
from sklearn.utils import resample
#create two different dataframe of majority and minority class
dataset_majority = dataset[(dataset['Heart_Disease']==0)]
dataset_minority = dataset[(dataset['Heart_Disease']==1)]
# upsample minority class
df_minority_upsampled = resample(dataset_minority,
                                 replace=True,    # sample with replacement
                                 n_samples= 283883, # to match majority class
                                 random_state=42)  # reproducible results
# Combine majority class with upsampled minority class
dataset_upsampled = pd.concat([df_minority_upsampled, dataset_majority])

In [6]:
X = dataset_upsampled.iloc[:, :-1].values
y = dataset_upsampled.iloc[:, -1].values

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [ ]:
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.tree import DecisionTreeClassifier
#Create an instance
classifier = BalancedBaggingClassifier(base_estimator=DecisionTreeClassifier(),
                                sampling_strategy='not majority',
                                replacement=False,
                                random_state=42)
classifier.fit(X_train, y_train)
preds = classifier.predict(X_test)

/usr/local/lib/python3.10/dist-packages/imblearn/ensemble/_bagging.py:353: FutureWarning: `base_estimator` was renamed to `estimator` in version 0.10 and will be removed in 0.12.
  warnings.warn(


In [ ]:
#print(preds)
from sklearn.metrics import confusion_matrix, accuracy_score,f1_score
cm = confusion_matrix(y_test, preds)
print(cm)

[[67985  3267]
 [   26 70664]]


In [ ]:
print(accuracy_score(y_test, preds))
print(f1_score(y_test,preds))

0.9768003832551324
0.9772301394679889


In [8]:
from sklearn.ensemble import RandomForestClassifier

In [9]:
classifier_rf = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 0)

In [10]:
classifier_rf.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', random_state=0)

In [11]:
y_pred = classifier_rf.predict(X_test)

In [12]:
from sklearn.metrics import confusion_matrix, accuracy_score,f1_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(accuracy_score(y_test, y_pred))
print(f1_score(y_test,y_pred))

[[68755  2497]
 [    9 70681]]
0.9823449014386157
0.9825812550393416


In [15]:
pip install pickle-mixin

  Preparing metadata (setup.py) ... done
  Created wheel for pickle-mixin: filename=pickle_mixin-1.0.2-py3-none-any.whl size=5991 sha256=9cde899ae4a5326d2992a013aab3ff1e286b305d274ea9e2155d0bab5047825b
  Stored in directory: /root/.cache/pip/wheels/3e/c6/e9/d1b0a34e1efc6c3ec9c086623972c6de6317faddb2af0a619c
Successfully built pickle-mixin


In [17]:
import pickle

In [21]:
with open('/model_pkl', 'wb') as files:
    pickle.dump(classifier_rf, files)

In [22]:
# load saved model
with open('model_pkl' , 'rb') as f:
    lr = pickle.load(f)

In [23]:
# check prediction

lr.predict([[0,0,0,1,0,72,150,32.66,14.54,1,0,30,16,12,0,0,0,1,0]]) # similar

array([0])

In [25]:
pip install flask_restful

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 1.6 MB/s eta 0:00:00


In [26]:
pip install flask_cors

In [27]:
from flask import Flask, request, redirect
from flask_restful import Resource, Api
from flask_cors import CORS
import os

In [28]:
import pickle
import pandas as pd
import json

def predict_mpg(config):
    ##loading the model from the saved file
    pkl_filename = "model.pkl"
    with open(pkl_filename, 'rb') as f_in:
        model = pickle.load(f_in)

    if type(config) == dict:
        df = pd.DataFrame(config)
    else:
        df = config

    y_pred = model.predict(df)

    if y_pred == 0:
        return 'Normal'
    elif y_pred == 1:
        return 'Alert Condition'

In [29]:
app = Flask(__name__)
cors = CORS(app, resources={r"*": {"origins": "*"}})
api = Api(app)

In [30]:
class Test(Resource):
    def get(self):
        return 'Welcome to, Test App API!'

    def post(self):
        try:
            value = request.get_json()
            if(value):
                return {'Post Values': value}, 201

            return {"error":"Invalid format."}

        except Exception as error:
            return {'error': error}

In [31]:
class GetPredictionOutput(Resource):
    def get(self):
        return {"error":"Invalid Method."}

    def post(self):
        try:
            data = request.get_json()
            predict = predict_mpg(data)
            predictOutput = predict
            return {'predict':predictOutput}

        except Exception as error:
            return {'error': error}

In [32]:
api.add_resource(Test,'/')
api.add_resource(GetPredictionOutput,'/getPredictionOutput')

In [36]:
pip install flask_ngrok

In [45]:
!pip install pyngrok==3.1.0


  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-3.1.0-py3-none-any.whl size=13550 sha256=2028a1038867c5c3f85c81082ac14c4e3b3a20909498b546e9939e870aa7c167
  Stored in directory: /root/.cache/pip/wheels/f5/fb/4f/be0dd46223d1c142cad946b9829cf3d0e7edd548f000df9cbb
Successfully built pyngrok
  Attempting uninstall: pyngrok
    Found existing installation: pyngrok 4.1.1
    Uninstalling pyngrok-4.1.1:
      Successfully uninstalled pyngrok-4.1.1


In [48]:
!pip install flask_ngrok

In [50]:
from flask_ngrok import run_with_ngrok
app = Flask(__name__)
run_with_ngrok(app)

In [49]:
!ngrok authtoken ""

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [51]:
app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://1a1b-34-118-250-29.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [06/Oct/2023 07:45:09] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [06/Oct/2023 07:45:10] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [06/Oct/2023 07:45:37] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [06/Oct/2023 07:45:38] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [06/Oct/2023 07:45:38] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [06/Oct/2023 07:45:44] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [06/Oct/2023 07:45:45] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [06/Oct/2023 07:45:46] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [06/Oct/2023 07:45:47] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [06/Oct/2023 07:45:47] "GET / HTTP/1.1" 404 -
